# An introduction to visualization with yt

[`yt`](https://yt-project.org) is a Python toolkit that provides a generic interface for analysis and visualization of spatially-organised data, with built-in compatibility for various data formats (Athena++, RAMSES, AREPO, to name a few).
It supports grid-based data (regular, streched, and various flavours of AMR), as well as particle data (point-like and SPH).
Its key features are:
- a code-agnostic interface (specificities of data formats are rarely relevant)
- dimensionally meaningful analysis
- lazy data loading (only load in memory what's actually needed, when it's needed)
- parallel capacity (with `mpi4py`)


In this tutorial, we'll demonstrate how to load data, and produce simple visulations with yt in a couple lines of Python, and show how to go beyond the defaults.
We will cover the following basic functionalities.
- `yt.load`
- `yt.SlicePlot`
- `yt.ProjectionPlot`
- `yt.Dataset.add_field`

Then, we will see how to enable compatibility with Idefix (and Pluto !) data files using the [`yt_idefix`](https://github.com/neutrinoceros/yt_idefix) extension.


For completness, `yt` supports many operations that will not be covered here, like
- volume rendering
- off-axis slices and projections
- region selection
- exporting reduced datasets to `HDF5`


In [ ]:
import yt

In [ ]:
ds = yt.load_sample("IsolatedGalaxy")
# ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030.hierarchy")

In [ ]:
yt.SlicePlot(ds, "z", ("gas", "density"))

In [ ]:
# plot the *column* density
yt.ProjectionPlot(ds, "z", ("gas", "density"))

In [ ]:
p = yt.SlicePlot(ds, "z", ("gas", "density"))
p

In [ ]:
p.zoom(30)

In [ ]:
p.annotate_timestamp(draw_inset_box=True)

In [ ]:
p.annotate_streamlines(("gas", "velocity_x"), ("gas", "velocity_y"), color="white")

In [ ]:
p.swap_axes()

In [ ]:
p.flip_horizontal()

In [ ]:
import unyt as un
import unyt.physical_constants as cst
import yt

ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030.hierarchy")

GM = cst.G * (10**6 * cst.Msun)
(GM * cst.Mearth / un.cm**2).units.dimensions

In [ ]:
def gravitational_pull(field, data):
    # the pull exerted by a central mass of 10**6 Msun on a test mass (Mearth)
    # as a function of the position
    xc, yc, zc = data.ds.domain_center
    length_unit = data["index", "x"].units
    d2 = (
        (data["index", "x"] - xc) ** 2
        + (data["index", "y"] - yc) ** 2
        + (data["index", "z"] - zc) ** 2
        + 1e-16 * length_unit**2  # smoothing around origin
    )
    return GM * un.Mearth / d2


ds.add_field(
    ("gas", "gravitational_pull"),
    function=gravitational_pull,
    sampling_type="cell",
    units="N",  # enable dimensional analysis check at definition time
    # force_override=True, # make this cell re-runable
)

p = yt.SlicePlot(ds, "x", ("gas", "gravitational_pull"))
p

In [ ]:
p.set_unit(("gas", "gravitational_pull"), "dyn")